In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Info'].tolist()

总数量：500


['https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304960',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304968',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304972',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304976',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658980',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304980',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304984',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15382989',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15382993',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722076',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304992',
 'https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570025',
 'https://www.rockauto.com/en/morei

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [17]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [27]:
a = 0

resp = requests.get(input_.loc[a, 'Info'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE HTML>\n<html class="no-js REPLACEHTMLCLASS" onclick="" lang="en">\n<head>\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n<link rel="stylesheet" type="text/css" href="/jslib/combined/css/desktopmoreinfo.css?1718191458">\n\n<script>\nif(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};\n_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift()

In [28]:
from bs4 import BeautifulSoup
from lxml import etree

In [29]:
# soup = BeautifulSoup(resp)
# html = etree.HTML(str(soup))
html = etree.HTML(str(resp))

# with open('./info.html', 'w', encoding='UTF-8') as file:
#     file.write(str(soup))

# print(soup.prettify())

In [30]:
html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() != 'Continue Shopping'

False

In [31]:
dict_info = {}
list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]
for i in range(len(list_tr)):
    dict_info[str(i)] = {}
    list_td = [str(_).strip() for _ in list_tr[i].xpath('./td/text()')]
    dict_info[str(i)][list_td[0]] = list_td[1]
    
dict_info

{'0': {'Camshaft Sprocket Material': 'High Alloy Steel'},
 '1': {'Keyway Adjustable': 'No'},
 '2': {'Sprockets Included': 'Yes'},
 '3': {'Timing Chain Type': 'Silent'}}

In [32]:
import json

In [33]:
info_json = json.dumps(dict_info)

info_json

'{"0": {"Camshaft Sprocket Material": "High Alloy Steel"}, "1": {"Keyway Adjustable": "No"}, "2": {"Sprockets Included": "Yes"}, "3": {"Timing Chain Type": "Silent"}}'

In [34]:
list_tbody = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or contains(@class, "altrow-b-1"))]')

list_tbody

[<Element tbody at 0x12a362b40>,
 <Element tbody at 0x12a588840>,
 <Element tbody at 0x12a699280>,
 <Element tbody at 0x12a698680>,
 <Element tbody at 0x12a698d00>,
 <Element tbody at 0x12a699100>,
 <Element tbody at 0x12a698d80>,
 <Element tbody at 0x12a698600>,
 <Element tbody at 0x12a698580>,
 <Element tbody at 0x12a698340>]

In [35]:
list_quantity = [int(tbody.xpath('./tr/td[1]')[0].text.strip()) for tbody in list_tbody]

list_quantity

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [36]:
list_note = [tbody.xpath('./tr/td[2]')[0].text.strip() for tbody in list_tbody]

list_note

['Timing Chain',
 'Timing Chain Guide / Damper',
 'Timing Chain Tensioner',
 'Timing Chain Guide / Damper',
 'Oil Pump Chain Tensioner',
 'Timing Crank Sprocket',
 'Timing Cam Sprocket',
 'Oil Pump Chain',
 'Oil Pump Chain Guide',
 'Oil Pump Sprocket']

In [37]:
list_json = [json.loads(tbody.xpath('./tr/td[contains(@id, "listing_data_essential")]/@value')[0]) for tbody in list_tbody]

list_json

[{'groupindex': '2',
  'carcode': 0,
  'parttype': '5724',
  'partkey': '8306448',
  'opts': []},
 {'groupindex': '4',
  'carcode': 0,
  'parttype': '5728',
  'partkey': '8307424',
  'opts': []},
 {'groupindex': '5',
  'carcode': 0,
  'parttype': '5736',
  'partkey': '8308132',
  'opts': []},
 {'groupindex': '6',
  'carcode': 0,
  'parttype': '5728',
  'partkey': '8308292',
  'opts': []},
 {'groupindex': '7',
  'carcode': 0,
  'parttype': '11860',
  'partkey': '8308296',
  'opts': []},
 {'groupindex': '8',
  'carcode': 0,
  'parttype': '5723',
  'partkey': '8314844',
  'opts': []},
 {'groupindex': '9',
  'carcode': 0,
  'parttype': '5722',
  'partkey': '8314860',
  'opts': []},
 {'groupindex': '10',
  'carcode': 0,
  'parttype': '5568',
  'partkey': '8527968',
  'opts': []},
 {'groupindex': '11',
  'carcode': 0,
  'parttype': '11859',
  'partkey': '8921276',
  'opts': []},
 {'groupindex': '12',
  'carcode': 0,
  'parttype': '5589',
  'partkey': '8921472',
  'opts': []}]

In [38]:
list_type_code = [json_['parttype'].strip() for json_ in list_json]

list_type_code

['5724',
 '5728',
 '5736',
 '5728',
 '11860',
 '5723',
 '5722',
 '5568',
 '11859',
 '5589']

In [39]:
list_part_key = [json_['partkey'].strip() for json_ in list_json]

list_part_key

['8306448',
 '8307424',
 '8308132',
 '8308292',
 '8308296',
 '8314844',
 '8314860',
 '8527968',
 '8921276',
 '8921472']

In [40]:
list_json_2 = [json.loads(tbody.xpath('./tr/input[contains(@id, "listing_data_supplemental")]/@value')[0]) for tbody in list_tbody]

list_json_2

[{'partnumber': '715F',
  'catalogname': 'MELLING',
  'belongstolisting': '3',
  'sortgroup': None,
  'sortgrouptext': None,
  'paramdesc': None,
  'iskitcomponent': True},
 {'partnumber': 'BG5515',
  'catalogname': 'MELLING',
  'belongstolisting': '3',
  'sortgroup': None,
  'sortgrouptext': None,
  'paramdesc': None,
  'iskitcomponent': True},
 {'partnumber': 'BT5514',
  'catalogname': 'MELLING',
  'belongstolisting': '3',
  'sortgroup': None,
  'sortgrouptext': None,
  'paramdesc': None,
  'iskitcomponent': True},
 {'partnumber': 'BT7015',
  'catalogname': 'MELLING',
  'belongstolisting': '3',
  'sortgroup': None,
  'sortgrouptext': None,
  'paramdesc': None,
  'iskitcomponent': True},
 {'partnumber': 'BT7016',
  'catalogname': 'MELLING',
  'belongstolisting': '3',
  'sortgroup': None,
  'sortgrouptext': None,
  'paramdesc': None,
  'iskitcomponent': True},
 {'partnumber': 'S899',
  'catalogname': 'MELLING',
  'belongstolisting': '3',
  'sortgroup': None,
  'sortgrouptext': None,
  

In [41]:
list_manufacturer = [json_['catalogname'].strip() for json_ in list_json_2]

list_manufacturer

['MELLING',
 'MELLING',
 'MELLING',
 'MELLING',
 'MELLING',
 'MELLING',
 'MELLING',
 'MELLING',
 'MELLING',
 'MELLING']

In [42]:
list_part_number = [json_['partnumber'].strip() for json_ in list_json_2]

list_part_number

['715F',
 'BG5515',
 'BT5514',
 'BT7015',
 'BT7016',
 'S899',
 'S906A',
 '717F',
 'BG7019',
 'S905']

In [43]:
dict_kit = {}
for i in range(len(list_tbody)):
    dict_kit[str(i)] = {}
    dict_kit[str(i)]['0'] = list_quantity[i]
    dict_kit[str(i)]['1'] = list_note[i]
    dict_kit[str(i)]['2'] = list_manufacturer[i]
    dict_kit[str(i)]['3'] = list_part_number[i]
    dict_kit[str(i)]['4'] = list_type_code[i]
    dict_kit[str(i)]['5'] = list_part_key[i]
        
dict_kit

{'0': {'0': 1,
  '1': 'Timing Chain',
  '2': 'MELLING',
  '3': '715F',
  '4': '5724',
  '5': '8306448'},
 '1': {'0': 1,
  '1': 'Timing Chain Guide / Damper',
  '2': 'MELLING',
  '3': 'BG5515',
  '4': '5728',
  '5': '8307424'},
 '2': {'0': 1,
  '1': 'Timing Chain Tensioner',
  '2': 'MELLING',
  '3': 'BT5514',
  '4': '5736',
  '5': '8308132'},
 '3': {'0': 1,
  '1': 'Timing Chain Guide / Damper',
  '2': 'MELLING',
  '3': 'BT7015',
  '4': '5728',
  '5': '8308292'},
 '4': {'0': 1,
  '1': 'Oil Pump Chain Tensioner',
  '2': 'MELLING',
  '3': 'BT7016',
  '4': '11860',
  '5': '8308296'},
 '5': {'0': 1,
  '1': 'Timing Crank Sprocket',
  '2': 'MELLING',
  '3': 'S899',
  '4': '5723',
  '5': '8314844'},
 '6': {'0': 1,
  '1': 'Timing Cam Sprocket',
  '2': 'MELLING',
  '3': 'S906A',
  '4': '5722',
  '5': '8314860'},
 '7': {'0': 1,
  '1': 'Oil Pump Chain',
  '2': 'MELLING',
  '3': '717F',
  '4': '5568',
  '5': '8527968'},
 '8': {'0': 1,
  '1': 'Oil Pump Chain Guide',
  '2': 'MELLING',
  '3': 'BG7019',

In [44]:
kit_json = json.dumps(dict_kit)

kit_json

'{"0": {"0": 1, "1": "Timing Chain", "2": "MELLING", "3": "715F", "4": "5724", "5": "8306448"}, "1": {"0": 1, "1": "Timing Chain Guide / Damper", "2": "MELLING", "3": "BG5515", "4": "5728", "5": "8307424"}, "2": {"0": 1, "1": "Timing Chain Tensioner", "2": "MELLING", "3": "BT5514", "4": "5736", "5": "8308132"}, "3": {"0": 1, "1": "Timing Chain Guide / Damper", "2": "MELLING", "3": "BT7015", "4": "5728", "5": "8308292"}, "4": {"0": 1, "1": "Oil Pump Chain Tensioner", "2": "MELLING", "3": "BT7016", "4": "11860", "5": "8308296"}, "5": {"0": 1, "1": "Timing Crank Sprocket", "2": "MELLING", "3": "S899", "4": "5723", "5": "8314844"}, "6": {"0": 1, "1": "Timing Cam Sprocket", "2": "MELLING", "3": "S906A", "4": "5722", "5": "8314860"}, "7": {"0": 1, "1": "Oil Pump Chain", "2": "MELLING", "3": "717F", "4": "5568", "5": "8527968"}, "8": {"0": 1, "1": "Oil Pump Chain Guide", "2": "MELLING", "3": "BG7019", "4": "11859", "5": "8921276"}, "9": {"0": 1, "1": "Oil Pump Sprocket", "2": "MELLING", "3": 

In [45]:
if len(list_tbody) == 0:
    df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                             'Kit_Json': '',
                             'Kit_No.': '',
                             'Kit_Quantity': '',
                             'Kit_Note': '',
                             'Kit_Manufacturer': '',
                             'Kit_Part_Number': '',
                             'Kit_Type_Code': '',
                             'Kit_Part_Key': '',
                             'Kit_JOIN_TCPK': '',
                             'Specifications_Json': info_json}])
    for tr in list_tr:
        df_temp.loc[0, tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

else:
    df_temp = pd.DataFrame({'Info': input_.loc[a, 'Info'],
                            'Kit_Json': kit_json,
                            'Kit_No.': [i+1 for i in range(len(list_tbody))],
                            'Kit_Quantity': list_quantity,
                            'Kit_Note': list_note,
                            'Kit_Manufacturer': list_manufacturer,
                            'Kit_Part_Number': list_part_number,
                            'Kit_Type_Code': list_type_code,
                            'Kit_Part_Key': list_part_key,
                            'Kit_JOIN_TCPK': [type_code + ';' + part_key for type_code, part_key in zip(list_type_code, list_part_key)],
                            'Specifications_Json': info_json})
    for tr in list_tr:
        df_temp[tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

10

In [46]:
crawler_status = 'ok'

crawler_status

'ok'

In [47]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [48]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info', 'Kit_No.'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

,Info,Kit_Json,Kit_No.,Kit_Quantity,Kit_Note,Kit_Manufacturer,Kit_Part_Number,Kit_Type_Code,Kit_Part_Key,Kit_JOIN_TCPK,Specifications_Json,camshaft sprocket material,keyway adjustable,sprockets included,timing chain type
0,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",1,1,Timing Chain,MELLING,715F,5724,8306448,5724;8306448,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
1,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",2,1,Timing Chain Guide / Damper,MELLING,BG5515,5728,8307424,5728;8307424,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
2,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",3,1,Timing Chain Tensioner,MELLING,BT5514,5736,8308132,5736;8308132,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
3,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",4,1,Timing Chain Guide / Damper,MELLING,BT7015,5728,8308292,5728;8308292,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
4,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",5,1,Oil Pump Chain Tensioner,MELLING,BT7016,11860,8308296,11860;8308296,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
5,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",6,1,Timing Crank Sprocket,MELLING,S899,5723,8314844,5723;8314844,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
6,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",7,1,Timing Cam Sprocket,MELLING,S906A,5722,8314860,5722;8314860,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
7,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",8,1,Oil Pump Chain,MELLING,717F,5568,8527968,5568;8527968,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
8,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",9,1,Oil Pump Chain Guide,MELLING,BG7019,11859,8921276,11859;8921276,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent
9,https://www.rockauto.com/en/moreinfo.php?pt=57...,"{""0"": {""0"": 1, ""1"": ""Timing Chain"", ""2"": ""MELL...",10,1,Oil Pump Sprocket,MELLING,S905,5589,8921472,5589;8921472,"{""0"": {""Camshaft Sprocket Material"": ""High All...",High Alloy Steel,No,Yes,Silent


In [49]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_info_error.xlsx', index=False)

output_error

""
